In [ ]:
from MILWRM.MxIF import *
from MILWRM.ST import *
from MILWRM.MILWRM import *

In [ ]:
# markers in COLON MAP PCA data
allmarkers = ["ACTININ","BCATENIN","CD3D","CD4_","CD8","CD11B","CD20","CD45","CD68",
              "CGA","COLLAGEN","COX2","DAPI","ERBB2","FOXP3","GAMMAACTIN","HLAA","LYSOZYME","MUC2",
              "NAKATPASE","OLFM4","SOX9","PANCK","PCNA","PEGFR","PSTAT3","SMA","SNA","VIMENTIN"]
eliotmarkers = ["VIMENTIN","SOX9","SMA","PSTAT3","PEGFR","PCNA","PANCK","OLFM4",
                "NAKATPASE","MUC5AC","MUC2","LYSOZYME","HLAA","GAMMAACTIN","FOXP3",
                "ERBB2","COLLAGEN","CGA","CDX2","CD68","CD45","CD20","CD11B","CD8",
                "CD4_","CD3D","BCATENIN","AQP5","COX2"]
channels = [
    "AQP5",
    "VIMENTIN",
    "SOX9",
    "SMA",
    "PSTAT3",
    "PEGFR",
    "PCNA",
    "PANCK",
    "OLFM4",
    "NAKATPASE",
    "MUC5AC",
    #"MUC2",
    "LYSOZYME",
    "HLAA",
    "GAMMAACTIN",
    "FOXP3",
    "CGA",
    "CDX2",
    "CD68",
    "CD45",
    "CD20",
    "CD11B",
    "CD8",
    "CD4_",
    "CD3D",
    "BCATENIN",
    
    #"CEACAM5",
    #"DPEP1",
    #"CD27",
    #"COX2",
    #"VIMENTIN",
    #"ERBB2",
    #"COLLAGEN",
]

In [ ]:
def create_tissue_mask_mxif(images, markers=None, downsample_factor=1):
    """
    Creates a whole tissue mask for the given tissue image

    Parameters
    ----------
    images : list or img
        images for which the tissue mask will be created for
    markers : list
        markers required to create MxIF labeller object

    Returns
    -------
    mask is added to self.mask
    """
    if isinstance(
        images, img
    ):  # force img objects into a list if a single object is given
        images = [images]
    if markers is None:  # using all markers if list of markers is not given
        markers = images[0].ch
    # TODO : check if the images have masks already or not
    for image in images:
        # create a copy of the image
        image_cp = image.copy()
        # creating a temprory mask
        w, h, d = image_cp.img.shape
        tmp_mask = np.ones((w, h))
        # setting the mask within the image object
        image_cp.mask = tmp_mask
        # creating the mask_tl tissue labeler
        mask_tl = mxif_labeler(images=[image_cp])
        # preprocessing before running MILWRM
        mask_tl.prep_cluster_data(
            features=markers, downsample_factor=downsample_factor, sigma=2, fract=0.1
        )
        # running MILWRM with two clusters
        mask_tl.label_tissue_regions(k=2, alpha=0.05)
        # estimating centroids
        scores = mask_tl.kmeans.cluster_centers_.copy()
        mean = mask_tl.kmeans.cluster_centers_.mean(axis=0)
        sd = mask_tl.kmeans.cluster_centers_.std(axis=0)
        # z-score cluster centroid values for more reliable loadings
        z_scores = (scores - mean) / sd
        # making sure the background is set as 0
        if z_scores[0].mean() > 0:
            print(z_scores[0], "the background is set as tissue ID 1")
            where_0 = np.where(mask_tl.tissue_IDs[0] == 0.0)
            mask_tl.tissue_IDs[0][where_0] = 0.5
            where_1 = np.where(mask_tl.tissue_IDs[0] == 1.0)
            mask_tl.tissue_IDs[0][where_1] = 0.0
            where_05 = np.where(mask_tl.tissue_IDs[0] == 0.5)
            mask_tl.tissue_IDs[0][where_05] = 1.0
        # rescaling the mask
        mask_final = resize(mask_tl.tissue_IDs[0], (w, h))
        # setting the final mask
        image.mask = mask_final

---
### HTA11_10711

In [ ]:
HTA11_10711 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S20_37543_1A/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; HTA11_10711

In [ ]:
HTA11_10711.clip()

In [ ]:
rgb = HTA11_10711.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(HTA11_10711, downsample_factor=14)

In [ ]:
rgb = HTA11_10711.show(ncols=5)

In [ ]:
%time HTA11_10711.to_npz("/Users/cody/Downloads/HTA11_10711_downsample8.npz")

---
### HTA11_06134

In [ ]:
HTA11_06134 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S20_8615_1A/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; HTA11_06134

In [ ]:
HTA11_06134.clip()

In [ ]:
rgb = HTA11_06134.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(HTA11_06134, downsample_factor=10)

In [ ]:
rgb = HTA11_06134.show(ncols=5)

In [ ]:
%time HTA11_06134.to_npz("/Users/cody/Downloads/HTA11_06134_downsample8.npz")

---
### HTA11_07862

In [ ]:
HTA11_07862 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S20_20146_2A/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; HTA11_07862

In [ ]:
HTA11_07862.clip()

In [ ]:
rgb = HTA11_07862.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(HTA11_07862, downsample_factor=10)

In [ ]:
rgb = HTA11_07862.show(ncols=5)

In [ ]:
%time HTA11_07862.to_npz("/Users/cody/Downloads/HTA11_07862_downsample8.npz")

---
### HTA11_07663

In [ ]:
HTA11_07663 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S20_19137_1A/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; HTA11_07663

In [ ]:
HTA11_07663.clip()

In [ ]:
rgb = HTA11_07663.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(HTA11_07663, downsample_factor=22)

In [ ]:
rgb = HTA11_07663.show(ncols=5)

In [ ]:
%time HTA11_07663.to_npz("/Users/cody/Downloads/HTA11_07663_downsample8.npz")

---
### WD84594

In [ ]:
WD84594 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/WD54594/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; WD84594

In [ ]:
WD84594.clip()

In [ ]:
rgb = WD84594.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD84594, downsample_factor=24)

In [ ]:
rgb = WD84594.show(ncols=5)

In [ ]:
%time WD84594.to_npz("/Users/cody/Downloads/WD84594_downsample8.npz")

---
### WD84221

In [ ]:
WD84221 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/WD84221/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; WD84221

In [ ]:
WD84221.clip()

In [ ]:
rgb = WD84221.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD84221, downsample_factor=20)

In [ ]:
rgb = WD84221.show(ncols=5)

In [ ]:
%time WD84221.to_npz("/Users/cody/Downloads/WD84221_downsample8.npz")

---
### SR00001

In [ ]:
SR00001 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/HUMAN_NL/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; SR00001

In [ ]:
SR00001.clip()

In [ ]:
rgb = SR00001.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(SR00001, downsample_factor=20)

In [ ]:
rgb = SR00001.show(ncols=5)

In [ ]:
%time SR00001.to_npz("/Users/cody/Downloads/SR00001_downsample8.npz")

---
### WD84216

In [ ]:
WD84216 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/WD84216/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; WD84216

In [ ]:
WD84216.clip()

In [ ]:
rgb = WD84216.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD84216, downsample_factor=22)

In [ ]:
rgb = WD84216.show(ncols=5)

In [ ]:
%time WD84216.to_npz("/Users/cody/Downloads/WD84216_downsample8.npz")

---
### WD84602

In [ ]:
WD84602 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/WD84602/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; WD84602

In [ ]:
WD84602.clip()

In [ ]:
rgb = WD84602.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD84602, downsample_factor=18)

In [ ]:
rgb = WD84602.show(ncols=5)

In [ ]:
%time WD84602.to_npz("/Users/cody/Downloads/WD84602_downsample8.npz")

---
### WD84596

In [ ]:
WD84596 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/WD84596/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; WD84596

In [ ]:
WD84596.clip()

In [ ]:
rgb = WD84596.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD84596, downsample_factor=22)

In [ ]:
rgb = WD84596.show(ncols=5)

In [ ]:
%time WD84596.to_npz("/Users/cody/Downloads/WD84596_downsample8.npz")

---
### WD84226

In [ ]:
WD84226 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/WD84226/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; WD84226

In [ ]:
WD84226.clip()

In [ ]:
rgb = WD84226.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD84226, downsample_factor=18)

In [ ]:
rgb = WD84226.show(ncols=5)

In [ ]:
%time WD84226.to_npz("/Users/cody/Downloads/WD84226_downsample8.npz")

---
### vTMA 3-5

In [ ]:
vTMA = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/VISIUM_TMA_2/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; vTMA

In [ ]:
vTMA.clip()

In [ ]:
rgb = vTMA.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(vTMA, downsample_factor=18)

In [ ]:
rgb = vTMA.show(ncols=5)

In [ ]:
%time vTMA.to_npz("/Users/cody/Downloads/vTMA_003_004_005_downsample8.npz")

---
### SG00001

In [ ]:
channels = [
    "AQP5",
    "VIMENTIN",
    "SOX9",
    "SMA",
    "PSTAT3",
    "PEGFR",
    "PCNA",
    "PANCK",
    "OLFM4",
    "NAKATPASE",
    "MUC5AC",
    #"MUC2",
    "LYSOZYME",
    "HLAA",
    "GAMMAACTIN",
    "FOXP3",
    "CGA",
    "CDX2",
    "CD68",
    #"CD45",
    "CD20",
    "CD11B",
    "CD8",
    "CD4_",
    "CD3D",
    "BCATENIN",
    
    #"CEACAM5",
    #"DPEP1",
    #"CD27",
    #"COX2",
    #"VIMENTIN",
    #"ERBB2",
    #"COLLAGEN",
]

In [ ]:
SG00001 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S19_19913_1J/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; SG00001

In [ ]:
SG00001.clip()

In [ ]:
rgb = SG00001.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(SG00001, downsample_factor=18)

In [ ]:
rgb = SG00001.show(ncols=5)

In [ ]:
%time SG00001.to_npz("/Users/cody/Downloads/SG00001_downsample8.npz")

---
### SG00002

In [ ]:
SG00002 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S20_39567_1H/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; SG00002

In [ ]:
SG00002.clip()

In [ ]:
rgb = SG00002.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(SG00002, downsample_factor=20)

In [ ]:
rgb = SG00002.show(ncols=5)

In [ ]:
%time SG00002.to_npz("/Users/cody/Downloads/SG00002_downsample8.npz")

---
### SG00003

In [ ]:
SG00003 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S21_34686_1N/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; SG00003

In [ ]:
SG00003.clip()

In [ ]:
rgb = SG00003.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(SG00003, downsample_factor=18)

In [ ]:
rgb = SG00003.show(ncols=5)

In [ ]:
%time SG00003.to_npz("/Users/cody/Downloads/SG00003_downsample8.npz")

---
### SG00004

In [ ]:
channels = [
    "AQP5",
    "VIMENTIN",
    "SOX9",
    "SMA",
    "PSTAT3",
    "PEGFR",
    "PCNA",
    "PANCK",
    "OLFM4",
    "NAKATPASE",
    "MUC5AC",
    "MUC2",
    "LYSOZYME",
    "HLAA",
    "GAMMAACTIN",
    "FOXP3",
    "CGA",
    "CDX2",
    "CD68",
    "CD45",
    "CD20",
    "CD11B",
    "CD8",
    "CD4_",
    "CD3D",
    "BCATENIN",
    
    #"CEACAM5",
    #"DPEP1",
    #"CD27",
    #"COX2",
    #"ERBB2",
    #"COLLAGEN",
]

In [ ]:
SG00004 = img.from_tiffs(
    tiffdir = "/Volumes/CellDive/scan_alpha/S18_12464_1H/AFRemoved/",
    common_strings = ["_stitched_downsample8.tif"],
    channels = channels,
) ; SG00004

In [ ]:
SG00004.clip()

In [ ]:
rgb = SG00004.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(SG00004, downsample_factor=20)

In [ ]:
rgb = SG00004.show(ncols=5)

In [ ]:
%time SG00004.to_npz("/Users/cody/Downloads/SG00004_downsample8.npz")